In [6]:
import os
import glob
from collections import Counter
import yaml

def analyze_yolo_dataset(dataset_path):
    """
    Анализирует датасет YOLO и выводит информацию о нем
    """
    print("=" * 50)
    print("АНАЛИЗ ДАТАСЕТА YOLO")
    print("=" * 50)
    
    # Проверяем существование папок
    folders = ['train', 'valid', 'val', 'test']
    existing_folders = []
    
    for folder in folders:
        if os.path.exists(os.path.join(dataset_path, folder)):
            existing_folders.append(folder)
    
    if not existing_folders:
        print("❌ Папки train/valid/test не найдены!")
        return
    
    print(f"📁 Найдены папки: {existing_folders}")
    print()
    
    # Ищем файл с классами (data.yaml или classes.txt)
    classes_file = None
    class_names = []
    
    # Пробуем найти data.yaml
    data_yaml_path = os.path.join(dataset_path, 'data.yaml')
    if os.path.exists(data_yaml_path):
        classes_file = data_yaml_path
        print(f"📄 Найден файл с классами: data.yaml")
        
        # Читаем YAML файл
        with open(data_yaml_path, 'r', encoding='utf-8') as file:
            data = yaml.safe_load(file)
        
        if 'names' in data:
            class_names = data['names']
            print(f"🎯 Классы из data.yaml: {class_names}")
        else:
            print("⚠️ В data.yaml не найдены имена классов (names)")
    
    # Если не нашли data.yaml, ищем classes.txt
    if not class_names:
        classes_txt_path = os.path.join(dataset_path, 'classes.txt')
        if os.path.exists(classes_txt_path):
            classes_file = classes_txt_path
            print(f"📄 Найден файл с классами: classes.txt")
            
            with open(classes_txt_path, 'r', encoding='utf-8') as file:
                class_names = [line.strip() for line in file.readlines() if line.strip()]
            print(f"🎯 Классы из classes.txt: {class_names}")
    
    if not class_names:
        print("⚠️ Файл с классами не найден, буду определять классы из аннотаций")
    
    # Анализируем каждую папку
    total_stats = {}
    
    for folder in existing_folders:
        print(f"\n📊 Анализ папки '{folder}':")
        print("-" * 30)
        
        folder_path = os.path.join(dataset_path, folder)
        images_path = os.path.join(folder_path, 'images')
        labels_path = os.path.join(folder_path, 'labels')
        
        # Проверяем существование подпапок
        if not os.path.exists(images_path):
            print(f"❌ Папка 'images' не найдена в {folder}")
            continue
            
        if not os.path.exists(labels_path):
            print(f"❌ Папка 'labels' не найдена в {folder}")
            continue
        
        # Считаем изображения
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        image_files = []
        
        for ext in image_extensions:
            image_files.extend(glob.glob(os.path.join(images_path, ext)))
            image_files.extend(glob.glob(os.path.join(images_path, ext.upper())))
        
        # Считаем аннотации
        label_files = glob.glob(os.path.join(labels_path, "*.txt"))
        
        print(f"🖼️  Изображений: {len(image_files)}")
        print(f"📝 Аннотаций: {len(label_files)}")
        
        # Анализируем классы в аннотациях
        folder_classes = Counter()
        total_objects = 0
        empty_files = 0
        
        for label_file in label_files:
            try:
                with open(label_file, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    
                if not lines:
                    empty_files += 1
                    continue
                    
                for line in lines:
                    line = line.strip()
                    if line:
                        parts = line.split()
                        if parts:
                            class_id = int(parts[0])
                            folder_classes[class_id] += 1
                            total_objects += 1
                            
            except Exception as e:
                print(f"⚠️ Ошибка чтения файла {label_file}: {e}")
        
        print(f"🎯 Объектов всего: {total_objects}")
        print(f"📦 Файлов без объектов: {empty_files}")
        
        # Выводим распределение по классам
        if folder_classes:
            print("📊 Распределение по классам:")
            for class_id, count in sorted(folder_classes.items()):
                class_name = class_names[class_id] if class_names and class_id < len(class_names) else f"class_{class_id}"
                percentage = (count / total_objects) * 100
                print(f"   {class_id}: {class_name} - {count} объектов ({percentage:.1f}%)")
        
        # Сохраняем статистику
        total_stats[folder] = {
            'images': len(image_files),
            'labels': len(label_files),
            'objects': total_objects,
            'classes': dict(folder_classes),
            'empty_files': empty_files
        }
    
    # Сводная статистика
    print("\n" + "=" * 50)
    print("📈 СВОДНАЯ СТАТИСТИКА")
    print("=" * 50)
    
    total_images = sum(stats['images'] for stats in total_stats.values())
    total_objects = sum(stats['objects'] for stats in total_stats.values())
    
    for folder, stats in total_stats.items():
        percentage = (stats['images'] / total_images) * 100 if total_images > 0 else 0
        print(f"📁 {folder}: {stats['images']} изображений ({percentage:.1f}%)")
    
    print(f"\n📊 Всего изображений: {total_images}")
    print(f"🎯 Всего объектов: {total_objects}")
    
    # Общее распределение по классам
    all_classes = Counter()
    for stats in total_stats.values():
        for class_id, count in stats['classes'].items():
            all_classes[class_id] += count
    
    if all_classes:
        print("\n🎯 ОБЩЕЕ РАСПРЕДЕЛЕНИЕ ПО КЛАССАМ:")
        for class_id, count in sorted(all_classes.items()):
            class_name = class_names[class_id] if class_names and class_id < len(class_names) else f"class_{class_id}"
            percentage = (count / total_objects) * 100
            print(f"   {class_id}: {class_name} - {count} объектов ({percentage:.1f}%)")

# Использование
if __name__ == "__main__":
    # Укажите путь к вашему датасету
    dataset_path = "/home/maxim/DS/projects/Mask-RCNN/datasets/Ships"
    
    # Замените на актуальный путь
    # dataset_path = "c:/datasets/my_yolo_dataset"
    # dataset_path = "./my_dataset"
    
    analyze_yolo_dataset(dataset_path)

АНАЛИЗ ДАТАСЕТА YOLO
📁 Найдены папки: ['train', 'valid', 'test']

📄 Найден файл с классами: data.yaml
🎯 Классы из data.yaml: ['ship']

📊 Анализ папки 'train':
------------------------------
🖼️  Изображений: 9697
📝 Аннотаций: 9697
🎯 Объектов всего: 19456
📦 Файлов без объектов: 351
📊 Распределение по классам:
   0: ship - 19456 объектов (100.0%)

📊 Анализ папки 'valid':
------------------------------
🖼️  Изображений: 2165
📝 Аннотаций: 2165
🎯 Объектов всего: 3720
📦 Файлов без объектов: 68
📊 Распределение по классам:
   0: ship - 3720 объектов (100.0%)

📊 Анализ папки 'test':
------------------------------
🖼️  Изображений: 1573
📝 Аннотаций: 1573
🎯 Объектов всего: 2872
📦 Файлов без объектов: 51
📊 Распределение по классам:
   0: ship - 2872 объектов (100.0%)

📈 СВОДНАЯ СТАТИСТИКА
📁 train: 9697 изображений (72.2%)
📁 valid: 2165 изображений (16.1%)
📁 test: 1573 изображений (11.7%)

📊 Всего изображений: 13435
🎯 Всего объектов: 26048

🎯 ОБЩЕЕ РАСПРЕДЕЛЕНИЕ ПО КЛАССАМ:
   0: ship - 26048 объектов

In [7]:
!pwd

/home/maxim/DS/projects/Mask-RCNN/notebooks/ships
